## 1. 환결 설정

 - 패키지 설치: albumentations(데이터 증강용), segmentation-models-pytorch, pycocotools 등
   필수 라이브러리를 설치

 - 장치 설정: 사용 가능한 최고의 하드웨어 가속기(cuda: NVIDIA GPU, mps: Mac silicon,
   또는 cpu)를 자동으로 선택


## 2. 데이터 탐색(EDA)

 - COCO 분석:
   - 이미지 및 어노테이션 수
   - 클래스 분포 (6개 클래스 식별: Undrivable, Road, Lane Mark, My bike, Rider, Moveable)
 - 원본 데이터 시각화: 샘플 이미지와 기존 마스크를 오버레이하여 데이터의 무결성을 확인


## 3. 데이터셋 구현

 - 클래스: MotorcycleSegDataset(Dataset)
   - 목적: Motorcycle Night Ride 데이터셋의 특정 디렉토리 구조를 처리하기 위한 커스텀 PyTorch Dataset
   - __init__: 이미지 및 새로 생성된 processed_masks의 파일 경로를 내부 변수로 저장
   - __getitem__:
     1. 이미지 로드 (BGR을 RGB로 변환)
     2. 마스크 로드 (픽셀 강도 = 클래스 ID인 그레이스케일 이미지)
     3. 증강 (Augmentation): albumentations를 사용하여 변환(리사이즈, ImageNet 통계로 정규화, 텐서 변환)을 이미지와 마스크에 동시에 적용하여 공간적 일관성을 유지
   - 출력: 텐서 쌍(image, mask)을 반환
 - DataLoader:
   - MotorcycleSegDataset을 훈련(Training) 및 검증(Validation) 로더로 래핑
   - 배치 처리(batch_size)와 데이터 셔플링(shuffling)을 관리


### 파트 1: 준비 단계
 - [패키지 설치 및 임포트]
   - 내용: pip install ..., import ...
   - 설명: 작업에 필요한 도구상자를 엽니다. torch(AI 엔진), albumentations(이미지 변형 도구), pycocotools(데이터 분석 도구) 등을 가져옵니다. "그래픽 카드(GPU)"가 있는지 확인해서 학습 속도를 높일 준비도 여기서 합니다.
 - [데이터 탐색 (EDA)]
   - 내용: COCO API로 데이터 불러오기, 클래스 확인.
   - 설명: "우리한테 어떤 재료가 있지?" 확인하는 단계입니다. 사진이 몇 장인지, '오토바이', '도로', '장애물' 같은 분류 항목이 무엇인지 훑어봅니다.

### 파트 2: 재료 손질 (전처리)
 - [데이터셋 정의 (MotorcycleSegDataset)]
   - 내용: class MotorcycleSegDataset(Dataset): ...
   - 설명: AI에게 데이터를 떠먹여 주는 "숟가락"을 만듭니다. 원본 사진(입력)과 정답 마스크(정답)를 짝지어서 하나씩 꺼내주는 역할을 합니다. 이때 사진 크기를 조절하거나 색감을 살짝 바꿔서(Augmentation) AI가 더 강하게 훈련되도록 돕습니다.

 - [데이터 로더 (DataLoader)]
   - 내용: DataLoader(...)
   - 설명: 숟가락으로 뜬 밥을 컨베이어 벨트에 올립니다. 데이터를 한 번에 몇 개씩(Batch Size) 묶어서 모델에게 전달할지 정합니다.


## 🚀 모델 학습 전 코드 실행 흐름도 (Call Flow)

이 흐름은 코드가 위에서 아래로 실행될 때 실제로 호출되는 순서를 따릅니다.

### 1. 환경 설정 및 데이터 로딩 (Setup & Loading)
  1. os.listdir(IMAGE_DIR)
    - 역할: 이미지 폴더 내의 모든 파일 목록을 가져옵니다.
    - 설명: 원본 이미지, 마스크 이미지 등을 구분하기 위해 파일 이름들을 수집합니다.
  2. COCO(JSON_PATH) (pycocotools.coco.COCO)
    - 역할: COCO API 객체 생성.
    - 설명: JSON 파일(어노테이션)을 파싱하여 메모리에 로드하고, 데이터 검색을 위한 인덱스를 구축합니다.
  3. coco.getCatIds()
    - 역할: 카테고리 ID 목록 조회.
    - 설명: 데이터셋에 정의된 모든 클래스(오토바이, 도로 등)의 고유 ID를 가져옵니다.
  4. coco.loadCats(cat_ids)
    - 역할: 카테고리 상세 정보 로드.
    - 설명: ID에 해당하는 실제 이름('Rider', 'Road' 등) 정보를 가져옵니다.
  5. coco.getImgIds()
    - 역할: 전체 이미지 ID 목록 조회.
    - 설명: 데이터셋에 포함된 모든 이미지의 ID 리스트를 확보합니다.
  6. coco.getAnnIds()
    - 역할: 전체 어노테이션 ID 목록 조회.
    - 설명: (통계 출력용) 총 어노테이션 개수를 확인합니다.

### 2. 데이터 시각화 및 검증 (Visualization & EDA)
  1. coco.loadImgs(img_id) (반복문 내)
    - 역할: 특정 이미지 정보 로드.
    - 설명: 샘플 이미지의 파일명, 너비, 높이 정보를 가져옵니다.
  2. Image.open(...) (PIL.Image)
    - 역할: 이미지 파일 읽기.
    - 설명: 디스크에서 실제 이미지 파일을 열어 픽셀 데이터를 가져옵니다.
  3. create_class_mask(coco, img_id, cat_remap) (사용자 정의 함수)
    - 역할: [핵심] 특정 이미지에 대한 정답 마스크(Segmentation Mask) 생성.
    - 내부 호출:
      - coco.getAnnIds(imgIds=img_id): 해당 이미지의 어노테이션 ID 조회.
      - coco.loadAnns(ann_ids): 어노테이션 상세 정보(폴리곤 좌표 등) 로드.
      - coco.annToMask(ann): 폴리곤 좌표를 0과 1로 된 비트맵 마스크로 변환.
    - 설명: 여러 개의 객체 마스크를 하나의 클래스 인덱스 맵(0, 1, 2...)으로 합칩니다.
  4. plt.imshow(...) (matplotlib.pyplot)
    - 역할: 이미지 및 마스크 출력.
    - 설명: 원본 이미지와 생성된 마스크를 화면에 그려서 데이터가 올바른지 눈으로 확인합니다.

### 3. 데이터셋 구축 (Dataset Construction)
  1. train_test_split(...) (sklearn.model_selection)
    - 역할: 데이터 분할.
    - 설명: 전체 이미지 ID 리스트를 학습용(Train)과 검증용(Val)으로 8:2 비율로 나눕니다.
  2. A.Compose([...]) (albumentations)
    - 역할: 데이터 증강 파이프라인 정의.
    - 설명: Resize, HorizontalFlip, Normalize, ToTensorV2 등의 변환 과정을 하나로 묶습니다.
  3. MotorcycleSegDataset(...) (사용자 정의 클래스 - 인스턴스 생성)
    - 역할: 데이터셋 객체 생성 (train_dataset, val_dataset).
    - 설명: coco 객체, 이미지 경로, 변환기(transform) 등을 저장해 두고, 나중에 데이터를 요청받을 준비를 합니다. (이 시점에서는 데이터를 아직 읽지 않음)
  4. DataLoader(...) (torch.utils.data)
    - 역할: 데이터 로더 생성 (train_loader, val_loader).
    - 설명: 배치(Batch) 단위로 데이터를 묶고, 셔플링(Shuffle) 등의 기능을 제공하는 반복자(Iterator)를 만듭니다.

### 4. 데이터셋 동작 확인 (Verification)
  1. train_dataset[0] (인덱싱 호출)
    - 내부 호출: MotorcycleSegDataset.__getitem__(0)
    - 역할: 첫 번째 데이터 샘플 가져오기.
    - 동작 순서 (__getitem__ 내부):
      1. coco.loadImgs: 이미지 정보 로드.
      2. Image.open: 이미지 파일 읽기.
      3. coco.getAnnIds -> coco.loadAnns: 어노테이션 로드.
      4. coco.annToMask: 마스크 생성 및 합치기.
      5. self.transform(...): [증강 적용] 이미지 리사이즈, 뒤집기, 정규화 등 수행.
      6. return img, mask: 최종 텐서 반환.
    - 설명: 데이터셋 클래스가 잘 만들어졌는지, 첫 번째 데이터를 실제로 뽑아서 형상(Shape)과 값(Values)을 확인합니다.


## 🔍 요약 설명

이 코드의 흐름은 "준비 -> 확인 -> 조립 -> 최종 점검" 단계로 이루어져 있습니다.

1. 준비: COCO API를 통해 복잡한 JSON 데이터를 쉽게 다룰 수 있는 상태로 만듭니다.
2. 확인: create_class_mask 함수를 통해 데이터가 올바르게 마스크로 변환되는지 눈으로 봅니다.
3. 조립: MotorcycleSegDataset 클래스와 Albumentations 증강 도구를 결합하여, 학습 가능한 형태의 데이터를 만들어내는 공장(Dataset)을 세웁니다.
4. 최종 점검: 공장에서 샘플 하나를 꺼내(dataset[0]) 텐서 형태가 모델에 들어가기에 적합한지 확인합니다.


### 1. 🔍 전체적인 구조 확인 (Keys)

```
# COCO 객체가 가지고 있는 주요 속성들 (딕셔너리 키) 확인
print("COCO 객체 속성:", coco.__dict__.keys())

# 예시 출력:
# dict_keys(['dataset', 'anns', 'imgToAnns', 'catToImgs', 'imgs', 'cats'])
```


### 2. 🖼️ 이미지 데이터 확인 (coco.imgs)

```
# 첫 번째 이미지 정보 하나만 출력해 보기
first_img_id = coco.getImgIds()[0]
print(f"\n[이미지 ID {first_img_id}] 정보:")
print(coco.imgs[first_img_id])

# 예시 출력:
# {'license': 4, 'file_name': '000000397133.jpg', 'coco_url': '...', 'height': 427, 'width': 640, 'date_captured': '...', 'flickr_url': '...', 'id': 397133}
```

### 3. 🏷️ 카테고리 데이터 확인 (coco.cats)

```
# 첫 번째 카테고리 정보 하나만 출력해 보기
first_cat_id = coco.getCatIds()[0]
print(f"\n[카테고리 ID {first_cat_id}] 정보:")
print(coco.cats[first_cat_id])

# 예시 출력:
# {'supercategory': 'person', 'id': 1, 'name': 'person'}
```

### 4. 📝 어노테이션 데이터 확인 (coco.anns)

```
# 첫 번째 어노테이션 정보 하나만 출력해 보기
# (주의: 전체 어노테이션 딕셔너리는 매우 크므로 하나만 뽑아서 출력합니다.)
first_ann_id = list(coco.anns.keys())[0]
print(f"\n[어노테이션 ID {first_ann_id}] 정보:")
print(coco.anns[first_ann_id])

# 예시 출력:
# {'segmentation': [[125.12, 539.69, ...]], 'area': 702.1057, 'iscrowd': 0, 'image_id': 397133, 'bbox': [473.07, 395.93, 38.65, 28.67], 'category_id': 18, 'id': 1768}
```

### create_class_mask

1. 🎭 create_class_mask 함수
 - 역할: JSON 파일 안에 흩어져 있는 "좌표 정보(Annotation)"들을 모아 하나의 "그림(Segmentation Mask)"으로 만들어주는 함수입니다.
 - 비유: "점 잇기 그림책"에서 여러 점(좌표)들을 선으로 잇고 페인트칠까지 해서, 최종적으로 어떤 물체가 어디에 있는지 색깔별로 칠해진 하나의 완성된 도화지를 만들어내는 것과 같습니다.

2. 🛠️ 주요 용도
 1. 데이터 시각화 (눈으로 확인):
   - 우리가 가진 데이터가 제대로 된 데이터인지 확인하려면 그림으로 그려봐야 합니다.
   - JSON에는 [100, 200, 150, 250...] 같은 숫자들만 잔뜩 들어있어서 사람이 알아볼 수 없습니다.
   - 이 함수가 숫자를 그림으로 바꿔주면, 원본 사진 위에 덮어 씌워서 "아, 여기가 오토바이구나!" 하고 확인할 수 있게 해줍니다.
 2. 데이터 통계 분석 (분포 확인):
   - "도로가 전체 사진의 몇 %를 차지하지?", "오토바이는 얼마나 작게 나오지?" 등을 계산하려면, 면적(픽셀 수) 을 세어야 합니다.
   - 이 함수가 만든 마스크 이미지에서 0번 색깔(배경) 픽셀 수, 1번 색깔(도로) 픽셀 수를 세면 데이터의 클래스 분포를 알 수 있습니다.
 3. 모델 학습용 데이터 생성 (훈련 준비):
   - AI 모델(Segmentation)은 "[숫자 좌표]"를 입력받는 게 아니라, "[정답 그림(마스크 이미지)]" 를 입력받아 학습합니다.
   - 모델에게 "이 사진의 정답은 이거야"라고 알려주기 위해, 매번 학습 때마다(또는 미리 저장해두기 위해) 원본 좌표 정보를 마스크 이미지로 변환해야 합니다.

요약
create_class_mask 함수는 "숫자로 된 정답지(JSON)"를 "AI가 이해할 수 있는 그림 답안지(Mask)"로 번역해 주는 번역기입니다.<br />
모델 학습 전에는 "데이터가 정상인지 확인(시각화/통계)"하는 용도로 쓰이고, 모델 학습 중에는 "실제 정답 데이터를 모델에 공급(Dataset)"하는 핵심 부품으로 쓰입니다.


### 1. 샘플 시각화 (3장)
데이터셋의 처음, 중간, 끝 부분에서 각각 1장씩, 총 3장의 이미지를 뽑아 시각화합니다.
 - sample_ids = [...]: 전체 이미지 ID 리스트(img_ids)에서 0번(처음), 1/3 지점, 2/3 지점에 있는 이미지 ID 3개를 선택합니다.
 - plt.subplots(3, 3, ...): 3행 3열의 그리드 형태인 그래프(Canvas)를 준비합니다. (행: 이미지 3장, 열: 원본/마스크/오버레이)

[반복문] (for i, img_id in enumerate(sample_ids):) 선택된 3개의 이미지에 대해 다음 과정을 반복합니다.

 1. 원본 이미지 출력 (axes[i, 0]):
  - coco.loadImgs(img_id): 이미지 파일명을 가져옵니다.
  - Image.open(...): 실제 이미지 파일을 엽니다.
  - imshow(img): 첫 번째 열에 원본 이미지를 그립니다.

 2. 클래스 마스크(정답) 출력 (axes[i, 1]):
  - create_class_mask(...): JSON 좌표를 해석하여 정답 마스크 이미지를 생성합니다.
  - imshow(mask, cmap='tab10'): 두 번째 열에 마스크를 그립니다. tab10 컬러맵을 사용하여 클래스별로 다른 색상으로 표시합니다.

 3. Fuse(오버레이) 이미지 출력 (axes[i, 2]):
  - __fuse.png 파일이 존재하는지 확인합니다. (이 파일은 원본과 마스크를 겹쳐놓은 미리보기 이미지입니다.)
  - 파일이 있다면 세 번째 열에 출력하여, 마스크가 원본 위에 제대로 씌워졌는지 확인합니다.

### 2. 클래스 분포 분석
"데이터 불균형" 문제를 확인하기 위해, 전체 데이터셋에서 각 클래스가 차지하는 픽셀 수를 셉니다.
 - class_pixel_counts = Counter(): 각 클래스(0, 1, 2...)별 픽셀 개수를 저장할 카운터를 초기화합니다.
 - total_pixels = 0: 전체 픽셀 수를 저장할 변수입니다.

[반복문] (for img_id in img_ids:) 모든 이미지에 대해 다음을 반복합니다. (시간이 좀 걸릴 수 있습니다 ⏳)

 1. 마스크 생성: create_class_mask로 현재 이미지의 마스크를 만듭니다.
 2. 픽셀 수 세기: np.unique(mask, return_counts=True)를 사용하여 마스크에 포함된 각 클래스 값과 그 개수를 셉니다.
 3. 누적: class_pixel_counts에 구한 개수를 더하고, total_pixels에도 더합니다.

[그래프 그리기]
 - class_ratios = {...}: 각 클래스의 픽셀 수를 전체 픽셀 수로 나누어 비율(%) 을 계산합니다.
 - ax.barh(...): 클래스 이름과 비율을 사용하여 가로 막대그래프를 그립니다. 어떤 클래스가 데이터의 대부분을 차지하는지(이 경우 보통 '도로'나 '배경'이 많음) 한눈에 볼 수 있습니다.

[텍스트 출력]
 - 비율이 높은 순서대로 정렬하여 콘솔에 텍스트로 클래스명: 00.00% 형태로 출력합니다.
 - 마지막으로 샘플 이미지의 해상도(img.size)도 출력해 줍니다.


## MotorcycleSegDataset 

1. 🔧 핵심 역할 (Role)
 - 데이터 읽기: 사용자가 요청한 번호(idx) 에 해당하는 이미지 파일과 정답 마스크를 디스크(폴더)에서 읽어옵니다.
 - 전처리 (변환): 읽어온 이미지와 마스크를 AI 모델이 학습할 수 있는 형태(Tensor) 로 변환하고, 필요하다면 데이터 증강(Albumentations) 도 수행합니다.
 - 쌍으로 반환: (이미지, 마스크)를 항상 세트로 묶어서 줍니다.

2. 🎯 용도 (Usage)
 1. PyTorch DataLoader와의 결합:
  - 파이토치의 학습 시스템(DataLoader)은 데이터를 배치(Batch) 단위로 묶어 모델에 넣어줘야 합니다.
  - 이때 DataLoader가 "데이터 하나 줘!"라고 요청하면, 실제로 데이터를 찾아서 가공해 주는 일을 이 클래스가 담당합니다.
 2. 커스텀 데이터셋 로직 구현:
  - COCO 형식의 JSON 파일과 이미지 폴더 구조를 이해하고, 이를 Segmentation 학습용 데이터 형태로 맞춤 제작하기 위해 사용됩니다.

 3. 🏁 목적 (Purpose)
  - 추상화 (Abstraction):
    - 모델 학습 코드(train_one_epoch) 입장에서는 데이터가 파일로 있는지, DB에 있는지, 메모리에 있는지 알 필요가 없습니다.
    - 그저 "이 데이터셋한테 달라고 하면 (이미지, 정답)을 준다" 는 약속을 지키기 위한 인터페이스입니다.
  - 일관성 유지:
    - 모든 학습 데이터에 대해 동일한 전처리(리사이즈, 정규화 등) 를 자동으로 적용하여 모델 학습의 안정성을 보장합니다.


### 코드의 독립성과 사용 목적의 차이

1. 목적의 분리 (Decoupling):
  - create_class_mask: 데이터셋 클래스를 만들지 않고도, COCO 객체만 있으면 즉시 마스크를 확인해 보고 싶을 때 씁니다. (가볍고 빠름)
  - Dataset 클래스: 모델 학습을 위해 데이터 로더와 연동되어야 하며, 전처리 파이프라인(Transform)이 포함되어야 합니다. (무겁고 복잡함)

2. 수정의 편의성:
  - 만약 "시각화용 마스크는 원본 크기로 보고 싶고", "학습용 마스크는 512x512로 줄여서 쓰고 싶다"면?
  - 두 로직이 분리되어 있으므로 각각 다르게 설정하기 편합니다.

3. 일반적인 관행 (Common Practice):
  - 보통 utils.py 같은 곳에 공통 함수(def make_mask(...))를 하나 만들어두고, 양쪽에서 이를 가져다 쓰는 방식(재사용) 이 가장 깔끔합니다.
  - 하지만 이 노트북 예제에서는 사용자가 코드를 한눈에 파악하기 좋게 하기 위해, 혹은 개발 과정에서 실험적으로 코드를 짜다가 편의상 로직을 복사해서 두 군데에 적었을 가능성이 큽니다.

요약
  - 현재 상태: 기능적으로 문제는 없지만, 유지보수 측면에서 코드 중복이 발생했습니다.
  - 개선 방법: create_class_mask 함수를 더욱 견고하게 만들고, Dataset 클래스 안에서도 self.create_class_mask(...) 처럼 이 함수를 호출해서 쓰도록 수정하면 코드 중복을 없앨 수 있습니다. (리팩토링)


| 특징    | create_class_mask 함수 | MotorcycleSegDataset 클래스 내 __getitem__ |
| ----- | ---------------------------- | ----------------------------------------------- |
| 목적    | 단발성 시각화 및 분석용                | 반복적인 모델 학습용                                     |
| 호출 방법 | 사용자가 함수를 직접 호출               | DataLoader가 내부적으로 자동 호출                         |
| 실행 시점 | 학습 전 (Before Training)       | 학습 중 (During Training)                          |
| 반환값   | 순수 NumPy 배열 (이미지)            | 전처리(Resize, Tensor 변환 등)된 Tensor                |
| 의존성   | 독립 함수 (어디서든 호출 가능)           | Dataset 클래스 인스턴스에 종속됨                           |